Import Necessary Libraries

In [279]:
import statsapi as api
import pandas as pd
import tempfile
import os
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


Input Player Name and Date

In [280]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
date = '2024-04-20'
year = int(date[:4])
print(year)
player_name = f'{first_name.lower()} {last_name.lower()}'


2024


fetch player_id 

In [281]:
player_info = api.lookup_player(player_name, season=year)
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
print(player_id)
print(team_id)

661388
158


Gather batter_hand, batter_age, , batter_strike_zone_top, batter_strike_zone_bottom

In [282]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
batter_age = person_info['people'][0]['currentAge']
batter_strike_zone_top = person_info['people'][0]['strikeZoneTop']
batter_strike_zone_bottom = person_info['people'][0]['strikeZoneBottom']
batter_starting_year = int(person_info['people'][0]['mlbDebutDate'][:4])
batter_year = year - batter_starting_year


print(batter_age)
print(batter_year)
print(batter_hand)
print(batter_strike_zone_top)
print(batter_strike_zone_bottom)

26
4
R
3.46
1.69


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand

In [283]:
game_info = api.schedule(date=date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

sPitcher_info = api.lookup_player(sPitcher_name)
sPitcher_id = sPitcher_info[0]['id']
sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

print(game_status)
print(game_pk)
print(opp_id)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_id)
print(home)

True
745190
138
571945
R
2889
False


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified 

In [284]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])

    if x.get('label') == 'First pitch':
        game_time = x['value']

    if x.get('label') == 'Umpires':
        ump_info = x['value']
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        print(hp_ump_id)

        else:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            proj_ump_ids = []
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_ump_ids.append(ump_id)
            print(proj_ump_ids)

            

print(temp)
print(wind_speed)
print(game_time)


427413
58
11
1:16 PM.


fetch player game data: 
- batting position

In [285]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])
        print (batting_position)


2


fetch game player stats
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [286]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    k = None
    bb = None


print(f'ab: {ab}')
print(f'h: {h}')
print(f'r: {r}')
print(f'rbi: {rbi}')
print(f'hr: {hr}')
print(f'k: {k}')
print(f'bb: {bb}')

    

ab: 6
h: 3
r: 2
rbi: 1
hr: 0
k: 1
bb: 0


Format player URL for finding streak averages

In [287]:
formatted_player_name = player_name.replace(' ', '-')
print(formatted_player_name)

streak_url = f'https://www.mlb.com/player/{formatted_player_name}-{player_id}'


william-contreras


Fetch Streak Averages Data

In [288]:
response = requests.get(streak_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

div = soup.find('div', class_='player-splits--last player-splits--last-x')

header = div.find('thead').find('tr')

headers = {}

for th in header.find_all('th'):
    stat_index = th['data-index']
    stat_header = th.find('span')
    stat_name = stat_header.text
    headers[stat_index] = stat_name

table_row = div.find('tbody').find_all('tr')

data = []

for row in table_row:

    row_data = {}
    for td in row.find_all('td'):
        stat_type = td['data-col']
        stat = td.find('span')
        stat_value = stat.text
        row_data[headers[stat_type]] = stat_value
    
    if row_data:
        data.append(row_data)


Set Streak Averages to Appropriate Variable

In [289]:
last_7_data = data[0]
last_15_data = data[1]
last_30_data = data[2]

last_7_stats = {}
last_15_stats = {}
last_30_stats = {}

last_7_stats['ab_last_7'] = round((int(last_7_data['AB']) / 7 ), 3)
last_7_stats['h_last_7'] = round((int(last_7_data['H']) / 7 ), 3)
last_7_stats['r_last_7'] = round((int(last_7_data['R']) / 7 ), 3)
last_7_stats['rbi_last_7'] = round((int(last_7_data['RBI']) / 7 ), 3)
last_7_stats['hr_last_7'] = round((int(last_7_data['HR']) / 7 ), 3)
last_7_stats['bb_last_7'] = round((int(last_7_data['BB']) / 7 ), 3)
last_7_stats['k_last_7'] = round((int(last_7_data['SO']) / 7 ), 3)
last_7_stats['ba_last_7'] = float(last_7_data['AVG'])
last_7_stats['obp_last_7'] = float(last_7_data['OBP'])
last_7_stats['slg_last_7'] = float(last_7_data['SLG'])

last_15_stats['ab_last_15'] = round((int(last_15_data['AB']) / 15 ), 3)
last_15_stats['h_last_15'] = round((int(last_15_data['H']) / 15 ), 3)
last_15_stats['r_last_15'] = round((int(last_15_data['R']) / 15 ), 3)
last_15_stats['rbi_last_15'] = round((int(last_15_data['RBI']) / 15 ), 3)
last_15_stats['hr_last_15'] = round((int(last_15_data['HR']) / 15 ), 3)
last_15_stats['bb_last_15'] = round((int(last_15_data['BB']) / 15 ), 3)
last_15_stats['k_last_15'] = round((int(last_15_data['SO']) / 15 ), 3)
last_15_stats['ba_last_15'] = float(last_15_data['AVG'])
last_15_stats['obp_last_15'] = float(last_15_data['OBP'])
last_15_stats['slg_last_15'] = float(last_15_data['SLG'])

last_30_stats['ab_last_30'] = round((int(last_30_data['AB']) / 30 ), 3)
last_30_stats['h_last_30'] = round((int(last_30_data['H']) / 30 ), 3)
last_30_stats['r_last_30'] = round((int(last_30_data['R']) / 30 ), 3)
last_30_stats['rbi_last_30'] = round((int(last_30_data['RBI']) / 30 ), 3)
last_30_stats['hr_last_30'] = round((int(last_30_data['HR']) / 30 ), 3)
last_30_stats['bb_last_30'] = round((int(last_30_data['BB']) / 30 ), 3)
last_30_stats['k_last_30'] = round((int(last_30_data['SO']) / 30 ), 3)
last_30_stats['ba_last_30'] = float(last_30_data['AVG'])
last_30_stats['obp_last_30'] = float(last_30_data['OBP'])
last_30_stats['slg_last_30'] = float(last_30_data['SLG'])


last_7_df = pd.DataFrame([last_7_stats])
last_15_df = pd.DataFrame([last_15_stats])
last_30_df = pd.DataFrame([last_30_stats])

streak_data_df = pd.concat([last_7_df, last_15_df, last_30_df], axis=1)

print(streak_data_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  bb_last_7  k_last_7  \
0      4.143     1.286     0.429       0.714        0.0      0.429     1.286   

   ba_last_7  obp_last_7  slg_last_7  ...  ab_last_30  h_last_30  r_last_30  \
0       0.31       0.375       0.345  ...       2.833        1.0      0.633   

   rbi_last_30  hr_last_30  bb_last_30  k_last_30  ba_last_30  obp_last_30  \
0        0.667       0.133       0.333      0.767       0.353        0.429   

   slg_last_30  
0        0.553  

[1 rows x 30 columns]


Fetch Batter Season Averages

In [290]:
season_info = api.player_stat_data(personId=player_id, group='[hitting]', type='yearByYear', sportId=1)

for season in season_info['stats']:
    if int(season['season']) == year:
        season_data = season['stats']

games_played = season_data['gamesPlayed']

season_stats = {}

#season_stats['ab_szn'] = season_data['atBats']
#season_stats['h_szn'] = season_data['hits']
#season_stats['r_szn'] = season_data['runs']
#season_stats['rbi_szn'] = season_data['rbi']
#season_stats['hr_szn'] = season_data['homeRuns']
#season_stats['bb_szn'] = season_data['baseOnBalls']
#season_stats['k_szn'] = season_data['strikeOuts']
#season_stats['ba_szn'] = season_data['avg']
#season_stats['obp_szn'] = season_data['obp']
#season_stats['slg_szn'] = season_data['slg']

season_stats['ab_per_game'] = round((season_data['atBats'] / games_played), 3)
season_stats['h_per_game'] = round((season_data['hits'] / games_played), 3)
season_stats['r_per_game'] = round((season_data['runs'] / games_played), 3)
season_stats['rbi_per_game'] = round((season_data['rbi'] / games_played), 3)
season_stats['hr_per_game'] = round((season_data['homeRuns'] / games_played), 3)
season_stats['bb_per_game'] = round((season_data['baseOnBalls'] / games_played), 3)
season_stats['k_per_game'] = round((season_data['strikeOuts'] / games_played), 3)

season_stats['ab_per_h'] = round((season_data['atBats'] / season_data['hits']), 3)
season_stats['ab_per_r'] = round((season_data['atBats'] / season_data['runs']), 3)
season_stats['ab_per_rbi'] = round((season_data['atBats'] / season_data['rbi']), 3)
season_stats['ab_per_hr'] = season_data['atBatsPerHomeRun']

season_stats_df = pd.DataFrame([season_stats])

print(season_stats_df)


   ab_per_game  h_per_game  r_per_game  rbi_per_game  hr_per_game  \
0        4.048       1.429       0.905         0.952         0.19   

   bb_per_game  k_per_game  ab_per_h  ab_per_r  ab_per_rbi ab_per_hr  
0        0.476       1.095     2.833     4.474        4.25     21.25  


Merge Season Averages and Streak Data

In [291]:
batter_season_stats_df = pd.concat([streak_data_df, season_stats_df], axis=1)

print(batter_season_stats_df)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  bb_last_7  k_last_7  \
0      4.143     1.286     0.429       0.714        0.0      0.429     1.286   

   ba_last_7  obp_last_7  slg_last_7  ...  h_per_game  r_per_game  \
0       0.31       0.375       0.345  ...       1.429       0.905   

   rbi_per_game  hr_per_game  bb_per_game  k_per_game  ab_per_h  ab_per_r  \
0         0.952         0.19        0.476       1.095     2.833     4.474   

   ab_per_rbi  ab_per_hr  
0        4.25      21.25  

[1 rows x 41 columns]


Fetch Starting Pitcher Season Data

In [292]:
season_info = api.player_stat_data(personId=sPitcher_id, group='[pitching]', type='yearByYear', sportId=1)

for season in season_info['stats']:
    if int(season['season']) == year:
        season_data = season['stats']

print(season_data)

games_started = season_data['gamesStarted']

sp_data = {}

sp_data['sp_games_started'] = games_started
sp_data['sp_pc_per_start'] = round((season_data['numberOfPitches'] / games_started), 3)
sp_data['sp_ip_per_start'] = round((float(season_data['inningsPitched']) / games_started), 3)
sp_data['sp_h_per_start'] = round((season_data['hits'] / games_started), 3)
sp_data['sp_r_per_start'] = round((season_data['runs'] / games_started), 3)
sp_data['sp_hr_per_start'] = round((season_data['homeRuns'] / games_started), 3)
sp_data['sp_bb_per_start'] = round((season_data['baseOnBalls'] / games_started), 3)
sp_data['sp_outs_per_start'] = round((season_data['outs'] / games_started), 3)
sp_data['sp_k_per_start'] = round((season_data['strikeOuts'] / games_started), 3)
sp_data['sp_er_per_start'] = round((season_data['earnedRuns'] / games_started), 3)
sp_data['sp_k_p'] = season_data['strikePercentage']
sp_data['sp_era'] = season_data['era']
sp_data['sp_whip'] = season_data['whip']
sp_data['sp_baa'] = season_data['avg']
sp_data['sp_obpa'] = season_data['obp']
sp_data['sp_slga'] = season_data['slg']
sp_data['sp_pc_per_batter'] = round((season_data['numberOfPitches']) / season_data['battersFaced'], 3)

sp_data_df = pd.DataFrame([sp_data])

print(sp_data_df)


{'gamesPlayed': 5, 'gamesStarted': 5, 'groundOuts': 26, 'airOuts': 29, 'runs': 19, 'doubles': 9, 'triples': 0, 'homeRuns': 5, 'strikeOuts': 20, 'baseOnBalls': 7, 'intentionalWalks': 0, 'hits': 33, 'hitByPitch': 1, 'avg': '.311', 'atBats': 106, 'obp': '.353', 'slg': '.538', 'ops': '.891', 'caughtStealing': 0, 'stolenBases': 1, 'stolenBasePercentage': '1.000', 'groundIntoDoublePlay': 3, 'numberOfPitches': 406, 'era': '6.49', 'inningsPitched': '26.1', 'wins': 1, 'losses': 3, 'saves': 0, 'saveOpportunities': 0, 'holds': 0, 'blownSaves': 0, 'earnedRuns': 19, 'whip': '1.52', 'battersFaced': 116, 'outs': 79, 'gamesPitched': 5, 'completeGames': 0, 'shutouts': 0, 'strikes': 268, 'strikePercentage': '.660', 'hitBatsmen': 1, 'balks': 0, 'wildPitches': 0, 'pickoffs': 0, 'totalBases': 57, 'groundOutsToAirouts': '0.90', 'winPercentage': '.250', 'pitchesPerInning': '15.42', 'gamesFinished': 0, 'strikeoutWalkRatio': '2.86', 'strikeoutsPer9Inn': '6.84', 'walksPer9Inn': '2.39', 'hitsPer9Inn': '11.28', '

Format Batter vs Starting Pitcher URL

In [293]:
formatted_batter_name = player_name.replace(' ', '%20')

bvp_url = f'https://baseballsavant.mlb.com/illustrator?playerId={player_id}&playerType=batter&name={formatted_batter_name}&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22{sPitcher_id}%22]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=[]&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7'

Fetch Batter vs Starting Pitcher Stats

In [297]:
bvp_raw_data = {}
bvp_data = {}

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get(bvp_url)

driver.implicitly_wait(10)

stats_boxes = driver.find_elements(By.CLASS_NAME, 'stats-box')

for stats_box in stats_boxes:
    stat_type = stats_box.find_element(By.CLASS_NAME, 'stats-title')
    stat_value = stats_box.find_element(By.CLASS_NAME, 'stats-value')
    bvp_raw_data[stat_type.text] = stat_value.text


driver.quit()

bvp_data['vs_sp_pa'] = int(bvp_raw_data['PA'])
bvp_data['vs_sp_p_per_pa'] = round((int(bvp_raw_data['Pitches']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_h_per_pa'] = round((int(bvp_raw_data['H']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_hr_per_pa'] = round((int(bvp_raw_data['HR']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_k_per_pa'] = round((int(bvp_raw_data['SO']) / bvp_data['vs_sp_pa']), 3)
bvp_data['vs_sp_ba_per_pa'] = float(bvp_raw_data['AVG'])
bvp_data['vs_sp_k_p'] = round((float(bvp_raw_data['K%']) / 100), 3)

bvp_data_df = pd.DataFrame([bvp_data])

print(bvp_data_df)



   vs_sp_pa  vs_sp_p_per_pa  vs_sp_h_per_pa  vs_sp_hr_per_pa  vs_sp_k_per_pa  \
0         3           4.667           0.333              0.0           0.333   

   vs_sp_ba_per_pa  vs_sp_k_p  
0            0.333      0.333  


Merge Pitcher data and BvP data

In [298]:
sPitcher_stats_df = pd.concat([sp_data_df, bvp_data_df], axis=1)

print(sPitcher_stats_df)

   sp_games_started  sp_pc_per_start  sp_ip_per_start  sp_h_per_start  \
0                 5             81.2             5.22             6.6   

   sp_r_per_start  sp_hr_per_start  sp_bb_per_start  sp_outs_per_start  \
0             3.8              1.0              1.4               15.8   

   sp_k_per_start  sp_er_per_start  ... sp_obpa sp_slga sp_pc_per_batter  \
0             4.0              3.8  ...    .353    .538              3.5   

  vs_sp_pa vs_sp_p_per_pa vs_sp_h_per_pa  vs_sp_hr_per_pa  vs_sp_k_per_pa  \
0        3          4.667          0.333              0.0           0.333   

   vs_sp_ba_per_pa  vs_sp_k_p  
0            0.333      0.333  

[1 rows x 24 columns]


Format URL for Batter vs Opp Data
- If batter has not yet playyed Opp, omit this section from the stats.
- Otherwise, provide per game data for Opp.

Fetch Batter vs Opp Stats

Fetch Umpire Stats
- If Home Plate Ump announced
    - Return data for only HP ump 
- If Home Plate Ump not yet announced
    - Fetch data for all 4 umps
    - hp_ump stats = avg of 4 umps stats

Fetch Venue Stats *WIP